In [ ]:
import monai
import pydicom
import pathlib
from glob import glob
import matplotlib.pyplot as plt
# import pyxnat
import torch
import xnat

# Get the bundle name

In [ ]:
bundles_list = monai.bundle.get_all_bundles_list()
name, date = next((n, d) for n, d in bundles_list if 'lung' in n)

# Download the bundle
```python
python -m monai.bundle download --name spleen_ct_segmentation --bundle_dir "./"
```

In [ ]:
import pathlib
bundle_dir = pathlib.Path('../../bundles').resolve()
bundle_dir.mkdir(exist_ok=True)

monai.bundle.download(name=name, bundle_dir=bundle_dir)

In [ ]:
model_weights = monai.bundle.load(name=name, bundle_dir=bundle_dir)

In [ ]:
# use pretrained Lung Nodule model in monai package

from monai.apps.utils import download_and_extract
from monai.networks.nets.densenet import DenseNet121


monai.apps.detection()

# model = DenseNet121(spatial_dims=3, in_channels=1, out_channels=3)
# model.load_state_dict(torch.load("/path/to/downloaded/model/model.pt"))

In [ ]:
import monai
from monai.networks.nets.densenet import DenseNet121

model_name = "lung_nodule_detection_3d_ct"
model_hash = "fb8dbe001f55a03ed0d2c9d5b31dde8782f79669"

model = DenseNet121(
    pretrained=True,
    progress=True,
    model_name=model_name,
    model_hash=model_hash
)

# Connecting to XNAT's REST API 

In [ ]:
mode = 'local_machine'
server_dict = dict(local_machine='http://localhost:80' , inside_xnat='http://xnat-web:8080')
interface = pyxnat.Interface(server=server_dict[mode], user='admin', password='admin')

# Level 1. Project

In [ ]:
print('list of projects:', interface.select.projects().get())

In [ ]:
project_ID = 'demo1'
project = interface.select.project(project_ID)


print('list of users:', project.users())
print('list of subjects:', project.subjects().get())
print('list of experiments:', project.experiments().get())
print('list of resources:', project.resources().get())

# Level 2. Subject

In [ ]:
ix = 0
subject_ID = project.subjects().get()[ix]
print('subject_ID:', subject_ID)

subject = project.subject(subject_ID)
subject

# Level 3. Experiments

### Setting the experiment

In [ ]:
experiment_ID = project.experiments().get()[ix]
print('experiment_ID:', experiment_ID)

experiment = subject.experiment(experiment_ID)
experiment

### Creating a new experiment

In [ ]:
# experiment.create(experiments='xnat:ctSessionData')

# experiment.attrs.mset({
#     'xnat:mrSessionData/coil' : 'head',
#     'xnat:mrSessionData/marker' : 'right',
# })

# Level 4. Scans

### List of scans in the experiment

In [ ]:
print('scans:', experiment.scans().get() )

In [ ]:
i_s = 0
scan_ID = experiment.scans().get()[i_s]
print('scan_ID:', scan_ID)

scan = experiment.scan(scan_ID)
scan

In [ ]:
scan.children()

In [ ]:
# # List all resources for the scan
# for resource in scan.resources().fetchall():
#     print(resource)
#     # print(resource.id(), resource.label(), resource.datatype())

# scan.resource('9').get()

### Creating a New Scan

In [ ]:
# scan.create(scans='xnat:ctScanData')

# # Setting scan attributes
# scan.attrs.mset({
#     'xnat:ctScanData/parameters/imageType' : 'dicom',
#     'xnat:ctScanData/series_description': 'something'
# })

# Adding DICOM files to a scan
# dicom_resource = scan.resource('DICOM')
# dicom_resource.put_dir()

# Level 5. Resources

In [ ]:
resource = scan.resource('DICOM')

In [ ]:
files =  resource.files()
attributes = files[0].attributes()
attributes

In [ ]:
URI = pathlib.Path(attributes['URI'])
URI

In [ ]:
# content = interface.get(uri=URI).content.decode()
# content

In [ ]:
# subject_id = 'your-subject-id'
# session_id = 'your-session-id'
# dicom_files = xnat.select('/projects/your-project-id/subjects/{}/experiments/{}/scans/*/resources/DICOM/files'.format(subject_id, session_id))

In [ ]:
# Downloading the whole resource (consists of multiple dicom files in the scan)
local_machine_dir = pathlib.Path('.')
path_list = resource.get(dest_dir=local_machine_dir, extract=True)
path_list[:5]

# Visualization

In [ ]:
!pip install GDCM pylibjpeg pylibjpeg-libjpeg

In [ ]:
path = path_list[0]
dcm_file = pydicom.dcmread(path)
dcm_file.pixel_array

# Visualize the dicom image

In [ ]:
fig = plt.figure()
plt.imshow(dcm_file.pixel_array, cmap=plt.cm.bone)

## Applying transformations using monai library

In [ ]:
im = dcm_file.pixel_array 

from monai.transforms import AdjustContrast
adjuster = AdjustContrast(.02)
im2 = adjuster(im)
# plt.imshow(im2, cmap=plt.cm.bone)

In [ ]:
dataset = pydicom.Dataset(pixel_array=im2) 
dcm_file.update(dataset)
plt.imshow(dcm_file.pixel_array, cmap=plt.cm.bone)

In [ ]:
new_name = URI.stem + '_adjusted' + URI.suffix
print('new_name' , new_name)

out_dir = local_machine_dir.joinpath(new_name)
print('out_dir',out_dir)

pydicom.dcmwrite(out_dir, dcm_file)

list(local_machine_dir.glob('*.dcm'))

In [ ]:
resource = interface.select.project(project_ID).subject(subject_ID).experiment(experiment_ID).scan(scan_ID).resource('DICOM')
# file1 = resource.file(URI.name)
# file1.get('temp.dcm')

resource.file(ID=new_name).insert(out_dir)
# resource.file(ID=new_name).put(out_dir)

In [ ]:
# interface.select.project(project_ID).subject(subject_ID).experiment(experiment_ID).resource('DICOM').files

In [ ]:
resource.get(dest_dir=local_machine_dir, extract=True)